<a href="https://colab.research.google.com/github/Yogeshkumar170201/Yogeshkumar170201/blob/master/MatchingActivities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ykdahiya","key":"d38b7d9f508bcc40efcc891626f72ca3"}'}

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d hasyimabdillah/workoutfitness-video

100% 4.31G/4.32G [00:36<00:00, 118MB/s]
100% 4.32G/4.32G [00:36<00:00, 128MB/s]


In [7]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [8]:

!unzip /content/workoutfitness-video.zip -d /content/datset

Archive:  /content/workoutfitness-video.zip
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_1.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_10.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_11.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_12.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_13.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_14.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_15.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_16.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_17.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_18.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_19.mp4  
  inflating: /content/datset/barbell biceps curl/barbell biceps curl_2.mp4  
  inflating: /content/

In [42]:
import cv2
import numpy as np

def extract_features(video_path, num_frames):
    # Load the video using OpenCV
    cap = cv2.VideoCapture(video_path)
    
    # Initialize empty list to store the features
    features = []
    
    # Loop through each frame of the video
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
        
        # Break the loop if there are no more frames
        if not ret:
            break
        
        # Resize the frame to 112x112 pixels
        frame = cv2.resize(frame, (112, 112))
        
        # Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Stack 3 copies of the grayscale frame to create a 3-channel image
        frame_3channel = np.stack((frame_gray,)*3, axis=-1)
        
        # Normalize the pixel values between -1 and 1
        frame_norm = (frame_3channel - 128) / 128
        
        # Append the normalized frame to the list of features
        features.append(frame_norm)
        
        # If we have collected enough frames, break out of the loop
        if len(features) == num_frames:
            break
    
    # Pad or truncate the frames to match the fixed number of frames
    if len(features) < num_frames:
        num_pad_frames = num_frames - len(features)
        padding = np.zeros((num_pad_frames, 112, 112, 3))
        features = np.concatenate((features, padding), axis=0)
    elif len(features) > num_frames:
        features = features[:num_frames]
    
    # Convert the list of features to a numpy array
    features = np.array(features)
    
    # Reshape the array to (1, num_frames, height, width, channels)
    features = np.reshape(features, (1, features.shape[0], 112, 112, 3))
    
    # Return the extracted features
    return features


In [125]:

video1_features = extract_features('/content/datset/deadlift/deadlift_25.mp4', 100)
video2_features = extract_features('/content/datset/deadlift/deadlift_10.mp4', 100)



In [126]:
print(video1_features.shape)
print(video2_features.shape)

(1, 100, 112, 112, 3)
(1, 100, 112, 112, 3)


In [127]:
from scipy.optimize import linear_sum_assignment
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np

def align_features(f1, f2):
    # Reshape the feature arrays
    f1 = np.reshape(f1, (f1.shape[0], -1))
    f2 = np.reshape(f2, (f2.shape[0], -1))

    # Compute the pairwise cosine distances between the features
    distances = pairwise_distances(f1, f2, metric='cosine')

    # Use the Hungarian algorithm to find the optimal assignment of features
    row_ind, col_ind = linear_sum_assignment(distances)

    # Sort the assignments in ascending order of row indices
    idx = np.argsort(row_ind)
    row_ind, col_ind = row_ind[idx], col_ind[idx]

    # Select the corresponding features from f1 and f2
    aligned_f1, aligned_f2 = f1[row_ind], f2[col_ind]

    return aligned_f1, aligned_f2


In [128]:
video1_aligned, video2_aligned = align_features(video1_features, video2_features)

In [129]:
print(video1_aligned.shape)
print(video2_aligned.shape)

(1, 3763200)
(1, 3763200)


In [130]:
from sklearn.metrics.pairwise import cosine_similarity

# Assuming you have already extracted and aligned features for both videos
# video1_aligned and video2_aligned

# Calculate the cosine similarity between the aligned features
similarity = cosine_similarity(video1_aligned, video2_aligned)

# Print the similarity score
print("Similarity between the two videos: ", similarity[0][0])


Similarity between the two videos:  0.8925429721228495


In [131]:
# # Load the pre-trained model
# model = load_model('activity_recognition_model.h5')

# # Extract features from video 1
# video1_features = extract_features('video1.mp4')

# # Extract features from video 2
# video2_features = extract_features('video2.mp4')

# # Align the features
# video1_aligned, video2_aligned = align_features(video1_features, video2_features)

# # Compare the aligned features
# predictions = model.predict([video1_aligned, video2_aligned])
# similarity_score = predictions[0][0]

# print("Similarity score between the two videos is:", similarity_score)
